## Groupby (Python y R)
En primer lugar, en términos generales, el programa pensión 65 se focaliza en hogares pobres con algún miembro del hogar mayor a 65 años. Usted debe crear una dummy si el hogar cumple tales características. Para ello use el módulo 2 (características de los miembros del hogar).

Primero, aplique groupby , luego realice un merge con el modulo 34, y finalmente, debe crear la variable dummy.

### "1.0 Set Directorio"

In [1]:
import pandas as pd
import numpy as np
import os # for usernanme y set directorio

### "2.0 Cargar dataset de ENAHO"

In [2]:
enaho_02 = pd.read_stata(r"../../enaho/2020/737-Modulo02/737-Modulo02/enaho01-2020-200.dta")
sumaria = pd.read_stata(r"../../enaho/2020/737-Modulo34/737-Modulo34/sumaria-2020.dta", convert_categoricals=False)

### "3.0 Mantener variables de interes"

In [3]:
enaho_02=enaho_02[['conglome','vivienda','hogar','p208a']]
sumaria=sumaria[['conglome','vivienda','hogar','pobreza','linea']]

### "4.0 Group_by"

In [4]:
enaho_02=enaho_02.groupby(['conglome','vivienda','hogar'],as_index = False)[['p208a']].max()
enaho_02


,conglome,vivienda,hogar,p208a
0,005002,011,11,66.0
1,005002,023,11,62.0
2,005002,049,11,74.0
3,005002,073,11,60.0
4,005002,097,11,80.0
...,...,...,...,...
34485,020353,023,11,42.0
34486,020353,030,11,32.0
34487,020353,049,11,59.0
34488,020353,055,11,47.0


#### Mantener la observación del adulto de mayor edad por hogar.
#### La lógica es que, si el individuo no es mayor de 65, entonces, ningún otro individuo del hogar lo será. 
#### Luego, si este individuo es mayor 65, el hogar ya puede ser clasificado en este requisito como 'apto' (pudiendo haber otras personas del hogar que también son mayores a 65 o no).

In [5]:
sumaria.keys() #confirmar cuáles variables python lee como tales para el merge

Index(['conglome', 'vivienda', 'hogar', 'pobreza', 'linea'], dtype='object')

In [6]:
enaho_02.keys() #confirmar cuáles variables python lee como tales para el merge

Index(['conglome', 'vivienda', 'hogar', 'p208a'], dtype='object')

### "5.0 Juntar bases"

In [7]:
enaho_2020=pd.merge(enaho_02,sumaria)
enaho_2020

,conglome,vivienda,hogar,p208a,pobreza,linea
0,005002,011,11,66.0,3,303.520996
1,005002,023,11,62.0,3,303.520996
2,005002,049,11,74.0,3,303.520996
3,005002,073,11,60.0,3,303.520996
4,005002,097,11,80.0,3,303.520996
...,...,...,...,...,...,...
34485,020353,023,11,42.0,3,254.304962
34486,020353,030,11,32.0,3,254.304962
34487,020353,049,11,59.0,3,254.304962
34488,020353,055,11,47.0,3,254.304962


### "6.0 Crear dummy que cumpla ambos requisitos"
    # Requisito 1: ser un hogar con adulto mayor a 65
        # p208a >= 65

    # Requisito 2: ser hogar pobre (o extremo pobre)
        # pobreza < 3 ==> pues 1 = extremo pobre y 2 = pobre

In [8]:
#si se cumplen esos 2 requisitos, poner 1 (candidato al programa); si no, 0
enaho_2020["accesitario"] = np.where((enaho_2020['p208a']>=65) & (enaho_2020['pobreza'] < 3),1,0)

In [9]:
enaho_2020["accesitario"].value_counts() #ver cantidad para cada uno de los valores de la dummy de accesitario

0    32816
1     1674
Name: accesitario, dtype: int64

In [10]:
enaho_2020.accesitario

0        0
1        0
2        0
3        0
4        0
        ..
34485    0
34486    0
34487    0
34488    0
34489    0
Name: accesitario, Length: 34490, dtype: int32